In [1]:
from time import time

import nltk
nltk.download('words')

import pandas as pd
import numpy as np

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

from data_utils import DataUtils
from model_utils import ModelUtils
from nlp_utils import NLPUtils

[nltk_data] Downloading package words to /home/mrugeles/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/mrugeles/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mrugeles/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mrugeles/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
modelUtils = ModelUtils()
dataUtils = DataUtils()
nlpUtils = NLPUtils()


In [3]:
X, Y, category_names = dataUtils.load_db_data('data/DisasterResponse.db')

sqlite:///data/DisasterResponse.db


In [4]:
%time X = nlpUtils.create_vector_model(X)

TfidfTransformer features: (10000, 17230)
CPU times: user 1min 1s, sys: 1min 40s, total: 2min 42s
Wall time: 2min 42s


In [5]:
from tqdm import tqdm
from textblob import Word

def spellcheck(word, threshold):
    value = Word(word).spellcheck()[0]
    if (value[1] < threshold):
        return f'-1'
    return value[0]
    
def clean_count_vector(matrix):
    start = time()
    columns_df = pd.DataFrame(list(matrix.columns), columns = ['feature'])
    columns_df['feature_spellcheck'] = columns_df['feature'].progress_apply(lambda word: spellcheck(word, 0.7))
    print(f'feature_spellcheck time: {time() - start}')

    start = time()
    drop_columns = columns_df.loc[columns_df['feature_spellcheck'] == "-1"]['feature'].values
    matrix = matrix.drop(drop_columns, axis = 1)
    print(f'drop_columns time: {time() - start}')


    start = time()
    renamed_columns = dict(columns_df.loc[columns_df['feature_spellcheck'] != "-1"].to_dict('split')['data'])
    matrix = matrix.rename(columns = renamed_columns)
    print(f'renamed_columns time: {time() - start}')

    start = time()
    for column in tqdm(matrix.columns[matrix.columns.duplicated()]):
        matrix[f'{column}_duplicated'] = matrix[column].groupby(level = 0, axis = 1).sum()
        matrix[f'{column}_duplicated'] = matrix[f'{column}_duplicated'].apply(lambda n: 1 if n > 1 else n)

    matrix = matrix.drop(matrix.columns[matrix.columns.duplicated()], axis = 1)

    matrix.rename(columns=lambda x: x.replace('_duplicated', ''), inplace=True)
    print(f'remove duplicated columns time: {time() - start}')

    return csr_matrix(matrix.values)

In [ ]:
tqdm.pandas(desc="feature_spellcheck")

X = clean_count_vector(X)

/home/mrugeles/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
feature_spellcheck:  55%|█████▌    | 9502/17230 [12:04<14:36,  8.82it/s]  